In [1]:
from huggingface_hub import hf_hub_download
import polars as pl

In [2]:
submission = pl.read_parquet('submission.parquet')

In [3]:
hf_hub_download(
    repo_id='deepvk/VK-LSVD', repo_type='dataset',
    filename='metadata/users_metadata.parquet', local_dir='VK-LSVD'
)

'VK-LSVD/metadata/users_metadata.parquet'

In [4]:
users_metadata = pl.read_parquet('VK-LSVD/metadata/users_metadata.parquet')
popular_users = users_metadata.sort('train_interactions_rank').select('user_id')
predictions = popular_users.to_numpy()[:len(submission)*100].reshape((len(submission), 100))

In [5]:
submission = submission.with_columns(user_id=predictions)
submission

item_id,user_id
u32,"array[u32, 100]"
3118,"[453232773, 86990582, … 116953252]"
26853,"[207024839, 310318706, … 102417890]"
41654,"[452381271, 348753378, … 185674961]"
55168,"[51348190, 413822332, … 140959458]"
65689,"[386411529, 442835260, … 70264092]"
…,…
608005402,"[24078335, 58545823, … 166836947]"
608030580,"[381790328, 213252880, … 159820542]"
608034282,"[303850133, 296112462, … 137956098]"


In [6]:
submission.write_parquet('submission.parquet')